# Few Shot: JMS -> SmallRye Reactive - ALL Incidents

> Work with a LLM to generate a fix for all of the common rule violations we find in our source app and the 'solved example'

We will include the solved example in this example to see if we get better quality results

##### Sample Applications Used
* 2 sample apps from [JBoss EAP Quickstarts](https://github.com/jboss-developer/jboss-eap-quickstarts/tree/7.4.x) were chosen:  helloworld-mdb & cmt
    * [helloworld-mdb](https://github.com/savitharaghunathan/helloworld-mdb)
    * [cmt](https://github.com/konveyor-ecosystem/cmt)

##### Using Custom Rules for JMS to SmallRye Reactive
* Rules were developed by Juanma [@jmle](https://github.com/jmle)
    * Rules originally from: https://github.com/jmle/rulesets/blob/jms-rule/default/generated/quarkus/05-jms-to-reactive-quarkus.windup.yaml


In [1]:
# Install local kai package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install -e ../../

Obtaining file:///Users/jmatthews/git/jwmatthews/kai
  Preparing metadata (setup.py) ... done
  Attempting uninstall: kai
    Found existing installation: kai 0.0.1
    Uninstalling kai-0.0.1:
      Successfully uninstalled kai-0.0.1
  Running setup.py develop for kai


In [2]:
# Bring in the common deps
import os
import pprint

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

pp = pprint.PrettyPrinter(indent=2)


def get_java_in_result(text: str):
    languages = ["java", "xml"]
    for lang in languages:
        _, updated_file = text.split(f"## Updated File")
        try:
            _, after = updated_file.split(f"```{lang}")
            return after.split("```")[0]
        except ValueError:
            # ValueError: not enough values to unpack (expected 2, got 1)
            print(f"Didn't find a result for language '{lang}'")
    print(f"Unable to find any Java or XML in the result:\n {text}")
    raise ValueError("Unable to parse result")


def write_output_to_disk(
    out_dir,
    formatted_template,
    example_javaee_file_contents,
    example_quarkus_file_contents,
    result,
):
    try:
        os.makedirs(out_dir, exist_ok=True)
    except OSError as error:
        print(f"Error creating directory {out_dir}: {error}")
        raise error

    # Save the template to disk
    template_path = os.path.join(out_dir, "template.txt")
    with open(template_path, "w") as f:
        f.truncate()
        f.write(formatted_template)
    print(f"Saved template to {template_path}")

    # Save the result
    result_path = os.path.join(out_dir, "result.txt")
    with open(result_path, "w") as f:
        f.truncate()
        f.write(result)
    print(f"Saved result to {result_path}")

    # Extract the updated java code and save it
    updated_file_contents = get_java_in_result(result)
    updated_file_path = os.path.join(out_dir, "updated_file.java")
    with open(updated_file_path, "w") as f:
        f.truncate()
        f.write(updated_file_contents)
    print(f"Saved updated java file to {updated_file_path}")

    # Save the original source code
    original_file_path = os.path.join(out_dir, "original_file.java")
    with open(original_file_path, "w") as f:
        f.truncate()
        f.write(example_javaee_file_contents)
    print(f"Saved original java file to {original_file_path}")

    # Save the Quarkus version we already in Git to use as a comparison
    known_quarkus_file_path = os.path.join(out_dir, "quarkus_version_from_git.java")
    with open(known_quarkus_file_path, "w") as f:
        f.truncate()
        f.write(example_quarkus_file_contents)
    print(f"Saved Quarkus version from Git to {known_quarkus_file_path}")

# Create a Prompt
## Step #1:  Create a Prompt Template


In [3]:
template_with_solved_example_and_analysis = """
    # Java EE to Quarkus Migration
    You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.
    I will give you an example of a JavaEE file and you will give me the Quarkus equivalent.

    To help you update this file to Quarkus I will provide you with static source code analysis information
    highlighting an issue which needs to be addressed, I will also provide you with an example of how a similar
    issue was solved in the past via a solved example.  You can refer to the solved example for a pattern of
    how to update the input Java EE file to Quarkus.

    Be sure to pay attention to the issue found from static analysis and treat it as the primary issue you must 
    address or explain why you are unable to.

    Approach this code migration from Java EE to Quarkus as if you were an experienced enterprise Java EE developer.
    Before attempting to migrate the code to Quarkus, explain each step of your reasoning through what changes 
    are required and why. 

    Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes 
    to imports we need to consider.

    As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.
    
    After you have shared your step by step thinking, provide a full output of the updated file:

    # Input information
    ## Issue found from static code analysis of the Java EE code which needs to be fixed to migrate to Quarkus
    Issue to fix:  "{analysis_message}"

    ## Solved Example Filename
    Filename: "{solved_example_file_name}"

    ## Solved Example Git Diff 
    This diff of the solved example shows what changes we made in past to address a similar problem.
    Please consider this heavily in your response.
    ```diff
    {solved_example_diff}
    ```

    ## Input file name
    Filename: "{src_file_name}"

    ## Input Line number of the issue first appearing in the Java EE code source code example below
    Line number: {analysis_lineNumber}
    
    ## Input source code file contents for "{src_file_name}"
    ```java 
    {src_file_contents}
    ```

    # Output Instructions
    Structure your ouput in Markdown format such as:

    ## Reasoning
    Write the step by step reasoning in this markdown section.
    If you are unsure of a step or reasoning, clearly state you are unsure and why.

    ## Updated File
    ```java
        Write the updated file for Quarkus in this section
    ```
   
    """

## Step #2: Find the common incidents we see in both applications

In [4]:
from kai.report import Report

ruleset_name = "custom-ruleset"


def find_common_rules(report1, report2, ruleset_name):
    common_rules = set(report1.report[ruleset_name]["violations"].keys()) & set(
        report2.report[ruleset_name]["violations"].keys()
    )
    return common_rules


# Static code analysis was run prior and committed to this repo
path_cmt_analysis = "./analysis_report/cmt/output.yaml"
path_helloworld_analysis = "./analysis_report/helloworld-mdb-quarkus/output.yaml"

# We will assume that helloworld is the app we want to migrate
# We will assume that CMT serves as our solved example

cmt_report = Report.load_report_from_file(path_cmt_analysis)
helloworld_report = Report.load_report_from_file(path_helloworld_analysis)

common_rule_names = find_common_rules(cmt_report, helloworld_report, ruleset_name)
common_rule_names

Reading report from ./analysis_report/cmt/output.yaml
Reading report from ./analysis_report/helloworld-mdb-quarkus/output.yaml


{'jms-to-reactive-quarkus-00000',
 'jms-to-reactive-quarkus-00010',
 'jms-to-reactive-quarkus-00020',
 'jms-to-reactive-quarkus-00030',
 'jms-to-reactive-quarkus-00050'}

## Step #3 Define a function to extract the analysis incident metadata

In [5]:
def get_incident_data(report, ruleset_name, rule):
    return report.report[ruleset_name]["violations"][rule]

## Step #4: Setup access to the source code in both applications via Git Repos


In [6]:
from kai.scm import GitDiff

##
# The App to MIGRATE will be HelloWorld MDB
helloworld_src_path = "../../kai_solution_server/samples/sample_repos/helloworld-mdb-quarkus"
helloworld_diff = GitDiff(helloworld_src_path)
# We checkout the quarkus branch so it's avaialble for us to consult, but we plan to use 'main'
helloworld_diff.checkout_branch("quarkus")
# Ensure we checked out the 'main' branch of the HelloWorld MDB repo
helloworld_diff.checkout_branch("main")

###
# SOLVED EXAMPLE will be CMT
###
# Setup access to look at the Git repo source code for each sample app
cmt_src_path = "../../kai_solution_server/samples/sample_repos/cmt"
cmt_diff = GitDiff(cmt_src_path)
# Ensure we checked out the 'quarkus' branch of the CMT repo
cmt_diff.checkout_branch("quarkus")

# Now we can extract the info we need
## We will assume:
## .   HelloWorld will be our source application to migrate, so we will use it's 'main' branch which maps to Java EE
## .   CMT will serve as our solved example, so we will consult it's 'quarkus' branch


## TODO
## Update here to use the common incidents and loop over all incidents
## Later we also want to lopo over other incidents

## Step 5: Define function to generate our prompt


In [7]:
def create_template_args_from_incident(
    cmt_report, helloworld_report, cmt_diff, hello_world_diff, ruleset_name, rule_name
):
    cmt_incident = get_incident_data(cmt_report, ruleset_name, rule_name)
    helloworld_incident = get_incident_data(helloworld_report, ruleset_name, rule_name)

    # We assume that we have at least one incident for each rule
    # will only use the first and ignore extras
    hw_entry = helloworld_incident["incidents"][0]
    src_file_name = helloworld_report.get_cleaned_file_path(hw_entry["uri"])
    src_file_contents = hello_world_diff.get_file_contents(src_file_name, "main")
    analysis_message = hw_entry["message"]
    analysis_lineNumber = hw_entry["lineNumber"]

    cmt_entry = cmt_incident["incidents"][0]
    solved_example_file_name = cmt_report.get_cleaned_file_path(cmt_entry["uri"])
    # solved_file_contents = cmt_diff.get_file_contents(solved_example_file_name, "quarkus")

    start_commit_id = cmt_diff.get_commit_from_branch("main").hexsha
    end_commit_id = cmt_diff.get_commit_from_branch("quarkus").hexsha

    solved_example_diff = cmt_diff.get_patch_for_file(
        start_commit_id, end_commit_id, solved_example_file_name
    )

    template_args = {
        "src_file_name": src_file_name,
        "src_file_contents": src_file_contents,
        "analysis_message": analysis_message,
        "analysis_lineNumber": analysis_lineNumber,
        "solved_example_file_name": solved_example_file_name,
        "solved_example_diff": solved_example_diff,
        # "solved_file_contents": solved_file_contents,
    }
    return template_args

# Step 6: Run through all the common rule violations

In [8]:
# model_name = "gpt-4-1106-preview"
model_name = "gpt-3.5-turbo-16k"
llm = ChatOpenAI(temperature=0.1, model_name=model_name)

common_rule_names = ["jms-to-reactive-quarkus-00000"]
for rule_name in common_rule_names:
    print(f"Processing rule: {rule_name}")
    template_args = create_template_args_from_incident(
        cmt_report,
        helloworld_report,
        cmt_diff,
        helloworld_diff,
        ruleset_name,
        rule_name,
    )
    formatted_prompt = template_with_solved_example_and_analysis.format(**template_args)
    prompt = PromptTemplate.from_template(template_with_solved_example_and_analysis)
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(template_args)

    src_file_name = template_args["src_file_name"]
    src_file_contents = template_args["src_file_contents"]
    # For comparison, we will look up what the source file looks like from Quarkus branch
    # this serves as a way of comparing to what the 'answer' is that was done manually by a knowledgeable developer
    src_file_from_quarkus_branch = helloworld_diff.get_file_contents(
        src_file_name, "quarkus"
    )

    output_dir = (
        f"output/{model_name}/helloworldmdb/{ruleset_name}/{rule_name}/few_shot/"
    )
    write_output_to_disk(
        output_dir,
        formatted_prompt,
        src_file_contents,
        src_file_from_quarkus_branch,
        result,
    )
    print(f"Completed processing of rule: {rule_name}")

Processing rule: jms-to-reactive-quarkus-00050
Saved template to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00050/few_shot/template.txt
Saved result to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00050/few_shot/result.txt
Saved updated java file to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00050/few_shot/updated_file.java
Saved original java file to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00050/few_shot/original_file.java
Saved Quarkus version from Git to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00050/few_shot/quarkus_version_from_git.java
Completed processing of rule: jms-to-reactive-quarkus-00050
Processing rule: jms-to-reactive-quarkus-00030
Saved template to output/gpt-3.5-turbo-16k/helloworldmdb/custom-ruleset/jms-to-reactive-quarkus-00030/few_shot/template.txt
Saved result to output/gpt-3.5-turbo-16k/he

ValueError: Unable to parse result

Should we consider iterating over fixes one at a time?
